In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd

In [3]:
#First need to get the flag results. 

df_f = pd.read_csv('FLAG_New_Columns.csv')
df_f.head()

,Unnamed: 0,index,CRN,ALCOHOL_RELATED,CELL_PHONE,DRINKING_DRIVER,DRUG_RELATED,AGGRESSIVE_DRIVING,SPEEDING_RELATED,COUNTY
0,0,0,2013016445,1,0,1,0,1,1,39
1,1,1,2012079040,0,0,0,0,1,1,39
2,2,2,2013016522,0,0,0,0,1,1,39
3,3,3,2013016456,0,0,0,0,1,1,39
4,4,4,2013016507,1,0,1,0,1,1,39


In [4]:
df_f.shape

#The flag dataset has already excluded the columns that we don't need

(46860, 10)

In [5]:
#Then I need the total crash results

df_c = pd.read_csv('file:///L:\lvpcgis\Transportation\Walk_BikeLV\
/Crash%20data\MunicipalityCrashData\CrashDataAnalysis\CrashData2012-2017.csv')

df_c.head()

C:\Users\bliu\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (16,17,31,32,33,34,36,38,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,OBJECTID,CRN,DISTRICT,COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,CRASH_MONTH,DAY_OF_WEEK,TIME_OF_DAY,...,BICYCLE_MAJ_INJ_COUNT,PED_COUNT,PED_DEATH_COUNT,PED_MAJ_INJ_COUNT,MAX_SEVERITY_LEVEL,COMM_VEH_COUNT,DEC_LAT,DEC_LONG,RDWY_SURF_TYPE_CD,MUNI_TXT
0,1,2017007557,5,48,48207,48207,2017,1,2,658,...,0,0,0,0,0,0,40.7850,-75.4957,,48207
1,2,2017021171,5,48,48214,68M06,2017,2,5,1635,...,0,0,0,0,0,0,40.8526,-75.1278,,48214
2,3,2017055050,5,48,48302,48302,2017,5,3,1327,...,0,0,0,0,0,0,40.6848,-75.2088,,48302
3,4,2017019828,5,48,48211,48211,2017,2,3,1501,...,0,0,0,0,0,0,40.7791,-75.3694,,48211
4,5,2017044292,5,48,48212,48212,2017,4,7,1201,...,0,0,0,0,9,0,40.7090,-75.2515,,48212


In [6]:
#Check what the error message is referring to
df_c.iloc[:, [16,17,31,32,33,34,36,38,88]].head()

#These columns are not important in our case, so I will just proceed

,WORK_ZONE_TYPE,WORK_ZONE_LOC,ARRIVAL_TM,CONS_ZONE_SPD_LIM,DISPATCH_TM,EST_HRS_CLOSED,LN_CLOSE_DIR,SPEC_JURIS_CD,RDWY_SURF_TYPE_CD
0,,,713,,700,3,6,,
1,,,1658,,1642,2,3,,
2,,,1327,,1327,1,1,,
3,,,1510,,1503,,,,
4,,,1208,,1201,,,,


In [7]:
#Pick the useful columns from df_c

df_c = df_c[['CRN', 'COUNTY', 'MUNICIPALITY', 'CRASH_YEAR', 'FATAL_COUNT',  'MAJ_INJ_COUNT']]

df_c.head()

,CRN,COUNTY,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT
0,2017007557,48,48207,2017,0,0
1,2017021171,48,48214,2017,0,0
2,2017055050,48,48302,2017,0,0
3,2017019828,48,48211,2017,0,0
4,2017044292,48,48212,2017,0,0


In [8]:
#This df_c doesn't have the municipality information so I need to join again

df_muni = pd.read_excel('file:///L:\lvpcgis\Transportation\
/Walk_BikeLV\Crash%20data\MunicipalityCrashData\CrashDataAnalysis\MuniExcel.xlsx')[['COUNTY', 'MUNICODE', 'NAME', 'NAMELABEL', 'CountyName', 'PDOT_MCode']]

df_muni = df_muni.rename(columns ={df_muni.columns[0]: 'LVPC_COUNTY'})

df_MuniCrash = pd.merge(df_c, df_muni.set_index('PDOT_MCode'), left_on = 'MUNICIPALITY', right_index = True)
df_muni.head()

df_MuniCrash.head()

#Try a different way of merging and see if the result is the same.

#Change the column name of the muni dataframe first
df_muni_v = df_muni.rename(columns = {df_muni.columns[-1]: 'MUNICIPALITY'})

df_Crash_vlookup = pd.merge(df_c, df_muni_v, how = 'left', on = 'MUNICIPALITY')

df_Crash_vlookup.head()

,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName,PDOT_MCode
0,77,1,ALBURTIS BOROUGH,ALBURTIS,Lehigh,39401
1,95,2,ALLEN TOWNSHIP,ALLEN,Northampton,48201
2,77,3,ALLENTOWN CITY,ALLENTOWN,Lehigh,39301
3,95,4,BANGOR BOROUGH,BANGOR,Northampton,48401
4,95,5,BATH BOROUGH,BATH,Northampton,48402


,CRN,COUNTY,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName
0,2017007557,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
15,2017006629,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
21,2017021000,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
44,2017006631,48,48207,2017,1,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
77,2017011650,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton


,CRN,COUNTY,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName
0,2017007557,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
1,2017021171,48,48214,2017,0,0,95,51,UPPER MT. BETHEL TOWNSHIP,UPPER MT. BETHEL,Northampton
2,2017055050,48,48302,2017,0,0,95,15,EASTON CITY,EASTON,Northampton
3,2017019828,48,48211,2017,0,0,95,34,MOORE TOWNSHIP,MOORE,Northampton
4,2017044292,48,48212,2017,0,0,95,39,PALMER TOWNSHIP,PALMER,Northampton


In [9]:
df_MuniCrash.shape
df_Crash_vlookup.shape

df_MuniCrash.head(10)

(46861, 11)

(46861, 11)

,CRN,COUNTY,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName
0,2017007557,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
15,2017006629,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
21,2017021000,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
44,2017006631,48,48207,2017,1,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
77,2017011650,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
79,2017021004,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
90,2017033687,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
96,2017025097,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
98,2017021002,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton
108,2017006630,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton


In [10]:
df_muni.columns[-1]

'PDOT_MCode'

In [11]:
#Now we can join the total crash data with the flag data
df_final_merge = pd.merge(df_MuniCrash, df_f, how = 'left', on = 'CRN')
df_final_merge.columns
df_final_merge.head(10)

Index(['CRN', 'COUNTY_x', 'MUNICIPALITY', 'CRASH_YEAR', 'FATAL_COUNT',
       'MAJ_INJ_COUNT', 'LVPC_COUNTY', 'MUNICODE', 'NAME', 'NAMELABEL',
       'CountyName', 'Unnamed: 0', 'index', 'ALCOHOL_RELATED', 'CELL_PHONE',
       'DRINKING_DRIVER', 'DRUG_RELATED', 'AGGRESSIVE_DRIVING',
       'SPEEDING_RELATED', 'COUNTY_y'],
      dtype='object')

,CRN,COUNTY_x,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName,Unnamed: 0,index,ALCOHOL_RELATED,CELL_PHONE,DRINKING_DRIVER,DRUG_RELATED,AGGRESSIVE_DRIVING,SPEEDING_RELATED,COUNTY_y
0,2017007557,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44410.0,643.0,0.0,0.0,0.0,0.0,1.0,0.0,48.0
1,2017006629,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44007.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0
2,2017021000,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44361.0,594.0,0.0,0.0,0.0,0.0,1.0,1.0,48.0
3,2017006631,48,48207,2017,1,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44120.0,353.0,1.0,0.0,1.0,0.0,1.0,1.0,48.0
4,2017011650,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44454.0,687.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0
5,2017021004,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44019.0,252.0,1.0,0.0,1.0,0.0,0.0,0.0,48.0
6,2017033687,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,46066.0,2299.0,0.0,0.0,0.0,0.0,1.0,1.0,48.0
7,2017025097,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44679.0,912.0,0.0,0.0,0.0,0.0,1.0,1.0,48.0
8,2017021002,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44035.0,268.0,0.0,0.0,0.0,0.0,1.0,0.0,48.0
9,2017006630,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44058.0,291.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0


In [14]:
#Change the name of the replicate column
df_final_merge = df_final_merge.rename(columns ={df_final_merge.columns[-1]: 'COUNTY_from_FLAG'})
df_final_merge.columns

Index(['CRN', 'COUNTY_x', 'MUNICIPALITY', 'CRASH_YEAR', 'FATAL_COUNT',
       'MAJ_INJ_COUNT', 'LVPC_COUNTY', 'MUNICODE', 'NAME', 'NAMELABEL',
       'CountyName', 'Unnamed: 0', 'index', 'ALCOHOL_RELATED', 'CELL_PHONE',
       'DRINKING_DRIVER', 'DRUG_RELATED', 'AGGRESSIVE_DRIVING',
       'SPEEDING_RELATED', 'COUNTY_from_FLAG'],
      dtype='object')

In [15]:
#Filter the column value to only 'Total'
crash1 = pd.read_excel('file:///L:\lvpcgis\Transportation\Walk_BikeLV\Crash%20data\
/MunicipalityCrashData\CrashDataAnalysis\TwoCyclesCrashData.xlsx', sheet_name = '2013-2017')

#Let's see if we can replace the NaN values like we intended
crash1[['MUNICODE','NAME', 'NAMELABEL']] = crash1[['MUNICODE','NAME', 'NAMELABEL']].fillna(method = 'ffill')
crash1.head(10)

,MUNICODE,NAME,NAMELABEL,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,MOD_INJ_COUNT,MIN_INJ_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_MAJ_INJ_COUNT,PED_DEATH_COUNT,PED_MAJ_INJ_COUNT
0,1.0,ALBURTIS BOROUGH,ALBURTIS,2013,0,0,0,0,0,0,0,0
1,1.0,ALBURTIS BOROUGH,ALBURTIS,2014,0,0,0,1,0,0,0,0
2,1.0,ALBURTIS BOROUGH,ALBURTIS,2015,0,0,0,7,0,0,0,0
3,1.0,ALBURTIS BOROUGH,ALBURTIS,2016,0,0,0,1,0,0,0,0
4,1.0,ALBURTIS BOROUGH,ALBURTIS,2017,1,0,0,0,0,0,0,0
5,1.0,ALBURTIS BOROUGH,ALBURTIS,Total,1,0,0,9,0,0,0,0
6,2.0,ALLEN TOWNSHIP,ALLEN,2013,0,3,3,14,0,0,0,0
7,2.0,ALLEN TOWNSHIP,ALLEN,2014,0,0,5,10,0,0,0,0
8,2.0,ALLEN TOWNSHIP,ALLEN,2015,0,1,3,14,0,0,0,0
9,2.0,ALLEN TOWNSHIP,ALLEN,2016,1,4,30,10,0,0,0,0


In [17]:
df = df_final_merge
df.columns

Index(['CRN', 'COUNTY_x', 'MUNICIPALITY', 'CRASH_YEAR', 'FATAL_COUNT',
       'MAJ_INJ_COUNT', 'LVPC_COUNTY', 'MUNICODE', 'NAME', 'NAMELABEL',
       'CountyName', 'Unnamed: 0', 'index', 'ALCOHOL_RELATED', 'CELL_PHONE',
       'DRINKING_DRIVER', 'DRUG_RELATED', 'AGGRESSIVE_DRIVING',
       'SPEEDING_RELATED', 'COUNTY_from_FLAG'],
      dtype='object')

In [18]:
df.head()

,CRN,COUNTY_x,MUNICIPALITY,CRASH_YEAR,FATAL_COUNT,MAJ_INJ_COUNT,LVPC_COUNTY,MUNICODE,NAME,NAMELABEL,CountyName,Unnamed: 0,index,ALCOHOL_RELATED,CELL_PHONE,DRINKING_DRIVER,DRUG_RELATED,AGGRESSIVE_DRIVING,SPEEDING_RELATED,COUNTY_from_FLAG
0,2017007557,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44410.0,643.0,0.0,0.0,0.0,0.0,1.0,0.0,48.0
1,2017006629,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44007.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0
2,2017021000,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44361.0,594.0,0.0,0.0,0.0,0.0,1.0,1.0,48.0
3,2017006631,48,48207,2017,1,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44120.0,353.0,1.0,0.0,1.0,0.0,1.0,1.0,48.0
4,2017011650,48,48207,2017,0,0,95,25,LEHIGH TOWNSHIP,LEHIGH,Northampton,44454.0,687.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0


In [19]:
#Better to create a list that includes all the columns that we need to use.
NewList = ['ALCOHOL_RELATED', 'DRINKING_DRIVER', 'SPEEDING_RELATED', 'AGGRESSIVE_DRIVING',
          'CELL_PHONE', 'DRUG_RELATED']

In [26]:
#Getting fatalities and major injuries with different flag tags

fatals = df[NewList].mul(df['FATAL_COUNT'], axis = 0).add_prefix('FATAL_')

df_flag_fatal = df.join(fatals)

#Do the same thing for major injuries.
ma_in = df[NewList].mul(df['MAJ_INJ_COUNT'], axis = 0).add_prefix('MAJ_INJ_')
df_flag_fama = df_flag_fatal.join(ma_in)

df_flag_fama.columns[-12:]

sum_cols = ['FATAL_COUNT', 'MAJ_INJ_COUNT'] + df_flag_fama.columns[-12:].tolist()

sum_cols


Index(['FATAL_ALCOHOL_RELATED', 'FATAL_DRINKING_DRIVER',
       'FATAL_SPEEDING_RELATED', 'FATAL_AGGRESSIVE_DRIVING',
       'FATAL_CELL_PHONE', 'FATAL_DRUG_RELATED', 'MAJ_INJ_ALCOHOL_RELATED',
       'MAJ_INJ_DRINKING_DRIVER', 'MAJ_INJ_SPEEDING_RELATED',
       'MAJ_INJ_AGGRESSIVE_DRIVING', 'MAJ_INJ_CELL_PHONE',
       'MAJ_INJ_DRUG_RELATED'],
      dtype='object')

['FATAL_COUNT',
 'MAJ_INJ_COUNT',
 'FATAL_ALCOHOL_RELATED',
 'FATAL_DRINKING_DRIVER',
 'FATAL_SPEEDING_RELATED',
 'FATAL_AGGRESSIVE_DRIVING',
 'FATAL_CELL_PHONE',
 'FATAL_DRUG_RELATED',
 'MAJ_INJ_ALCOHOL_RELATED',
 'MAJ_INJ_DRINKING_DRIVER',
 'MAJ_INJ_SPEEDING_RELATED',
 'MAJ_INJ_AGGRESSIVE_DRIVING',
 'MAJ_INJ_CELL_PHONE',
 'MAJ_INJ_DRUG_RELATED']

In [27]:
df_flag_fama.CRASH_YEAR.unique()
df_flag_fama.columns

array([2017, 2016, 2015, 2014, 2013, 2012], dtype=int64)

Index(['CRN', 'COUNTY_x', 'MUNICIPALITY', 'CRASH_YEAR', 'FATAL_COUNT',
       'MAJ_INJ_COUNT', 'LVPC_COUNTY', 'MUNICODE', 'NAME', 'NAMELABEL',
       'CountyName', 'Unnamed: 0', 'index', 'ALCOHOL_RELATED', 'CELL_PHONE',
       'DRINKING_DRIVER', 'DRUG_RELATED', 'AGGRESSIVE_DRIVING',
       'SPEEDING_RELATED', 'COUNTY_from_FLAG', 'FATAL_ALCOHOL_RELATED',
       'FATAL_DRINKING_DRIVER', 'FATAL_SPEEDING_RELATED',
       'FATAL_AGGRESSIVE_DRIVING', 'FATAL_CELL_PHONE', 'FATAL_DRUG_RELATED',
       'MAJ_INJ_ALCOHOL_RELATED', 'MAJ_INJ_DRINKING_DRIVER',
       'MAJ_INJ_SPEEDING_RELATED', 'MAJ_INJ_AGGRESSIVE_DRIVING',
       'MAJ_INJ_CELL_PHONE', 'MAJ_INJ_DRUG_RELATED'],
      dtype='object')

In [36]:
#Get the data for the two cycles.

df_cycle1 =  df_flag_fama.loc[df_flag_fama['CRASH_YEAR'] != 2017]


c1_r = df_cycle1.groupby(['NAMELABEL', 'MUNICODE'])[sum_cols].sum().reset_index()


In [37]:
c1_r

,NAMELABEL,MUNICODE,FATAL_COUNT,MAJ_INJ_COUNT,FATAL_ALCOHOL_RELATED,FATAL_DRINKING_DRIVER,FATAL_SPEEDING_RELATED,FATAL_AGGRESSIVE_DRIVING,FATAL_CELL_PHONE,FATAL_DRUG_RELATED,MAJ_INJ_ALCOHOL_RELATED,MAJ_INJ_DRINKING_DRIVER,MAJ_INJ_SPEEDING_RELATED,MAJ_INJ_AGGRESSIVE_DRIVING,MAJ_INJ_CELL_PHONE,MAJ_INJ_DRUG_RELATED
0,ALBURTIS,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALLEN,2,1,9,0.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0,4.0,5.0,0.0,1.0
2,ALLENTOWN,3,24,131,14.0,12.0,8.0,12.0,1.0,4.0,22.0,19.0,21.0,61.0,1.0,7.0
3,BANGOR,4,2,3,0.0,0.0,2.0,2.0,0.0,0.0,3.0,3.0,0.0,1.0,0.0,0.0
4,BATH,5,1,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,BETHLEHEM (LC),63,5,20,2.0,2.0,3.0,4.0,0.0,0.0,6.0,6.0,8.0,14.0,0.0,1.0
6,BETHLEHEM (NC),6,11,37,4.0,4.0,1.0,2.0,0.0,1.0,12.0,10.0,9.0,18.0,0.0,3.0
7,BETHLEHEM TWP.,7,11,16,5.0,4.0,6.0,7.0,0.0,3.0,1.0,1.0,5.0,8.0,0.0,2.0
8,BUSHKILL,8,6,15,3.0,2.0,3.0,4.0,0.0,0.0,9.0,9.0,9.0,9.0,0.0,0.0
9,CATASAUQUA,9,3,4,2.0,2.0,2.0,2.0,0.0,0.0,3.0,2.0,1.0,1.0,0.0,1.0


In [35]:
#Get the data for the two cycles.

#Most recent cycle

df_cycle2 =  df_flag_fama.loc[df_flag_fama['CRASH_YEAR'] != 2012]


c2_r = df_cycle2.groupby(['NAMELABEL', 'MUNICODE'])[sum_cols].sum().reset_index()

c2_r

,NAMELABEL,MUNICODE,FATAL_COUNT,MAJ_INJ_COUNT,FATAL_ALCOHOL_RELATED,FATAL_DRINKING_DRIVER,FATAL_SPEEDING_RELATED,FATAL_AGGRESSIVE_DRIVING,FATAL_CELL_PHONE,FATAL_DRUG_RELATED,MAJ_INJ_ALCOHOL_RELATED,MAJ_INJ_DRINKING_DRIVER,MAJ_INJ_SPEEDING_RELATED,MAJ_INJ_AGGRESSIVE_DRIVING,MAJ_INJ_CELL_PHONE,MAJ_INJ_DRUG_RELATED
0,ALBURTIS,1,1,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALLEN,2,2,8,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,4.0,5.0,0.0,1.0
2,ALLENTOWN,3,26,132,14.0,13.0,9.0,15.0,1.0,7.0,21.0,17.0,25.0,70.0,0.0,6.0
3,BANGOR,4,0,3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0,0.0,0.0
4,BATH,5,1,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,BETHLEHEM (LC),63,7,19,1.0,1.0,5.0,6.0,0.0,0.0,6.0,6.0,9.0,16.0,0.0,1.0
6,BETHLEHEM (NC),6,12,38,5.0,5.0,3.0,4.0,0.0,1.0,11.0,10.0,10.0,20.0,0.0,4.0
7,BETHLEHEM TWP.,7,14,16,6.0,4.0,6.0,7.0,0.0,3.0,1.0,1.0,7.0,9.0,0.0,2.0
8,BUSHKILL,8,5,14,3.0,2.0,2.0,3.0,0.0,0.0,5.0,5.0,7.0,8.0,0.0,1.0
9,CATASAUQUA,9,1,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#Save the result to excel

writer = pd.ExcelWriter('L:\lvpcgis\Transportation\Walk_BikeLV\Crash data\MunicipalityCrashData\CrashDataAnalysis\FlagDataForMapDec18.xlsx')
c1_r.to_excel(writer, 'Cycle1 Result')
c2_r.to_excel(writer, 'Cycle2 Result')
writer.save()

In [ ]:
df2 = df1.groupby(['NAME','ALCOHOL_RELATED'])['FATAL_COUNT', 'INJURY_COUNT'].sum().reset_index()
df2.columns

In [ ]:
#The solution provided by Stack Overflow
df = pd.DataFrame({'NAME': ['CityA', 'CityB', 'CityA', 'CityB', 'CityA', 'CityB'],
                   'FATAL#': [5, 5, 3, 3, 3, 2],
                   'INJURY#': [1, 1, 1, 1, 0, 2],
                   'ALCOHOL': [0, 0, 1, 1, 1, 0],
                   'CELL': [0, 1, 0, 0, 0, 0]})
fatals = 

df.iloc[:, -2:].mul(df['FATAL#'], axis = 0)
